# CSE 255 Programming Assignment 5

##  Problem Statement

In this programming assignment, you will estimate intrinsic dimension by calculating the Mean Squared Distance of the entire dataset to their representative centers. You will use the K-Means API in spark to find representative centers.
All of the necessary helper code is included in this notebook. However, we advise you to go over the lecture material, the EdX videos and the corresponding notebooks before you attempt this Programming Assignment.

## Reviewing the Theory 

### Computing the intrinsic dimension of a data set
Recall from class that given any $d$ dimensional dataset, we can divide it into $n$ cells of diameter $\epsilon$ each. The relationship between $n, d, \epsilon$ is then given by:
$$
n = \frac{C}{\epsilon^d}
$$
Where $C \in I\!R$

Alternately, we may write this as:
$$
\log{n} = \log{C} + d \times \log{\frac{1}{\epsilon}}
$$

Given this expression, we can then compute the dimensionality of a dataset using:
$$
d = \frac{\log{n_2} - \log{n_1}}{\log{\epsilon_1} - \log{\epsilon_2}}
$$



Where $(n_1,\epsilon_1)$, $(n_2, \epsilon_2)$ are the number of cells and diameter of each cell at 2 different scales.

### Using K-Means to estimate intrinsic dimension
We can use K-Means to approximate the value of intrinsic dimension of a data-set. In this case, the K centers represent the cells, each with a diameter equal to the Mean Squared Distance of the entire dataset to their representative centers. The estimate for intrinsic dimension then becomes:
$$
d = \frac{\log{n_2} - \log{n_1}}{\log{\sqrt{\epsilon_1}} - \log{\sqrt{\epsilon_2}}} = 2 \times \frac{\log{n_2} - \log{n_1}}{\log{\epsilon_1} - \log{\epsilon_2}}
$$

## Notebook Setup 

In [1]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import *
from math import log
import pickle

In [2]:
spark = SparkSession \
    .builder \
    .getOrCreate()
sc = spark.sparkContext

## Exercises

### Exercise 1: runKmeans

#### Example
The function <font color="blue">runKmeans</font> takes as input the complete dataset rdd, the sample of the rdd on which k-means needs to be run on, the k value and the count of elements in the complete data-set. It outputs the Mean Squared Distance(MSD) of all the points in the dataset from their closest centers, where the centers are calculated using k-means.

**<font color="magenta" size=2>Example Code</font>**
``` python
rdd = sc.parallelize([(1,1),(2,2),(3,3),(4,4),(5,5)])
runKmeans(rdd, sc.parallelize([(1,1),(2,2),(3,3)]), 3, rdd.count())
```

**<font color="blue" size=2>Example Output</font>**
``` python
2.0
```

<font color="red">**Hint : **</font> You might find [K-Means API](https://spark.apache.org/docs/2.2.0/mllib-clustering.html#k-means) useful. Ensure that the initializationMode parameter is set to kmeans++. The computeCost function gives you the sum of squared distances. Ensure you set maxIterations to 1 since we only need approximate values for the K centers to estimate intrinsic dimension. Setting this to higher values will lead to large execution times while barely affecting the calculation of the intrinsic dimension.

In [3]:
def runKmeans(data, sample_dataset, k, count):
    clusters = KMeans.train(sample_dataset, k, maxIterations=1)
    
    sum_of_squared_distances = clusters.computeCost(data)
    
    mean_squared_distances = sum_of_squared_distances/count
    
    return mean_squared_distances



In [61]:
KMeans

pyspark.mllib.clustering.KMeans

In [4]:
rdd = sc.parallelize([(1,1),(2,2),(3,3),(4,4),(5,5)])

In [5]:
runKmeans(rdd, sc.parallelize([(1,1),(2,2),(3,3)]), 3, rdd.count())

2.0

### Exercise 2: computeIntrinsicDimension

#### Example
The function <font color="blue">computeIntrinsicDimension</font> takes as input a pair of values $(n1, e1)$, $(n2, e2)$ and computes the intrinsic dimension as output. $e1, e2$ are the mean squared distances of data-points from their closest center

**<font color="magenta" size=2>Example Code</font>**
``` python
n1 = 10
n2 = 100
e1 = 10000
e2 = 100
computeIntrinsicDimension(n1, e1, n2, e2)
```

**<font color="blue" size=2>Example Output</font>**
``` python
1.0
```
<font color="red">**Hint : **</font> Use the last formula in the theory section 

In [4]:
def computeIntrinsicDimension(n1, e1, n2, e2):
    return 2*(log(n2)-log(n1))/(log(e1)-log(e2))


In [7]:
n1 = 10
n2 = 100
e1 = 10000
e2 = 100
computeIntrinsicDimension(n1, e1, n2, e2)

1.0

### Exercise 3: Putting it all together

#### Example
Now we run K-means for various values of k and use these to estimate the intrinsic dimension of the dataset. Since the dataset might be very large, running kmeans on the entire dataset to find k representative centers may take a very long time. To overcome this, we sample a subset of points from the complete dataset and run Kmeans only on these subsets. We will run Kmeans on 2 different subset sizes: 10000, 20000 points. We will be estimating the MSD for K values of 10, 200, 700, 2000.


The function <font color="blue">run</font> takes a dataframe containing the complete data-set as input and needs to do the following:
* For each sample size S
 * Take the first S elements from the dataframe
 * For each value of K (number of centroids)
  * Call runKmeans(data,S,K,data_count)
* Use the MSD values generated to calculate the intrinsic dimension where $(n_1, n_2) \in \{(10,200),(200,700),(700,2000)\}$.  

**NOTE: Ensure you the format of your output is identical to the one given below, i.e. the keys have to be of the format:**
```python
ID_<Subset_size>_<K-Value-1>_<K-Value-2>
```

**<font color="magenta" size=2>Example Code</font>**
``` python

df = spark.read.parquet(file_path)
run(df)
```

**<font color="blue" size=2>Example Output</font>**
``` python
{'ID_10000_10_200': 1.5574966096390015, 'ID_10000_200_700': 1.3064513902343675, 'ID_10000_700_2000': 1.091310378488035, 'ID_20000_10_200': 1.518279780870003, 'ID_20000_200_700': 1.2660237819996782, 'ID_20000_700_2000': 1.0151131917703071}
```
**Note: The output here is the output of the below function, i.e., the value stored in the variable where the 'run' function is called**

In [5]:
def run(df):
    output = {}
    for sample_size in [10000,20000]:
        last_k,last_msd = -1,-1
        for k in [10,200,700,2000]:
            msd = runKmeans(df,sc.parallelize(df.take(sample_size)),k,sample_size)
            if last_k == -1:
                last_k, last_msd = k, msd
                continue
            else:
                key = 'ID_%d_%d_%d'%(sample_size,last_k,k)
                val = computeIntrinsicDimension(last_k, last_msd, k, msd)
                output[key] = val
                last_k, last_msd = k, msd
    return output


In [6]:
# this function allows a +/-15% tolerance margin 
# while comparing student's answer against the solution
def within_tolerance(required_answer, student_answer, tolerance = 0.15):
            tolerance_value = required_answer * tolerance
            return required_answer - tolerance_value <= student_answer <= required_answer + tolerance_value

In [7]:
file_path_small = "hdfs://wahahr:9000/DSE23X/pa5/hw5-small.parquet"
df = spark.read.parquet(file_path_small)

In [8]:
df.head()

Row(col0='0.45152887379472995', col1='0.2260514832989532', col2='0.34592042250000005', col3='0.20345309649337506', col4='0.542974354273149', col5='0.2260514832989532', col6='0.34592042250000005', col7='0.20345309649337506')

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

## Spark 2.3
将dataFrame转为RDD，再用KMeans训练  
**方式一**  
```python
cols = df.columns

expr = [col(c).cast("Double").alias(c) 
        for c in df.columns]

df2 = df.select(*expr)
vectorAss = VectorAssembler(inputCols=cols, outputCol="features")
vdf = vectorAss.transform(df2)

data = vdf.rdd.map(lambda row: [x for x in row['features']])
```

**方式二**  
```python
data = df.rdd.map(lambda line: [float(x) for x in line])
```

## Spark 2.4
KMeans.fit 接受dataframe作为输入，只要frame中含有fearures字段，所以按照上诉方式一中的得到vdf后，就可以用来训练  
<img src="./Spark2.4.0-Estimator-input.jpg">

In [79]:
cols = df.columns

expr = [col(c).cast("Double").alias(c) 
        for c in df.columns]

df2 = df.select(*expr)
vectorAss = VectorAssembler(inputCols=cols, outputCol="features")
vdf = vectorAss.transform(df2)

data = vdf.rdd.map(lambda row: [x for x in row['features']])


In [10]:
data = df.rdd.map(lambda line: [float(x) for x in line])

In [11]:
print(data.getNumPartitions())

2


 WARN TaskSetManager: Stage 917 contains a task of very large size (325 KB). The maximum recommended task size is 100 KB.

In [140]:
%%timeit
res = run(data)

1 loop, best of 3: 56.7 s per loop


In [12]:
data5 = data.repartition(5)
print(data5.getNumPartitions())

5


 Stage 44 contains a task of very large size (147 KB). The maximum recommended task size is 100 KB.

In [13]:
%%timeit
res = run(data5)

1 loop, best of 3: 57.4 s per loop


In [14]:
data10 = data.repartition(10)
print(data10.getNumPartitions())

10


Stage 1054 contains a task of very large size (325 KB). The maximum recommended task size is 100 KB.

In [15]:
%%timeit
res = run(data10)

1 loop, best of 3: 53.7 s per loop


In [95]:
res

{'ID_10000_10_200': 1.5543801520031286,
 'ID_10000_200_700': 1.2698313797661345,
 'ID_10000_700_2000': 1.1282964749393567,
 'ID_20000_10_200': 1.5020515709327509,
 'ID_20000_200_700': 1.237115505288337,
 'ID_20000_700_2000': 1.0997170517303632}

In [96]:
assert within_tolerance(1.5528485676876376, res['ID_10000_10_200'])

In [97]:
assert within_tolerance(1.2563867109654632, res['ID_10000_200_700'])

In [98]:
assert within_tolerance(1.2041956732161911, res['ID_10000_700_2000'])

In [11]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [12]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [13]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [14]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [15]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [16]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [17]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [18]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [19]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [20]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


## Quiz based on PA 5 

**computeIntrinsicDimension**  
0.0/25.0 points (graded)  
What would computeIntrinsicDimension(700, 700, 200, 20000) return? (Round to 3 decimal places)    
**Answer: 0.7473811426956143**

In [99]:
computeIntrinsicDimension(700, 700, 200, 20000)

0.7473811426956143

**ID_20000_10_200**   
0.0/25.0 points (graded)
Following the PA5 notebook definitions, What value does ID_20000_10_200 have?  
**Answer: 1.564420312148605**  
**+/- 15% range:[1.3297572653263143,1.7990833589708959]**



In [100]:
res['ID_20000_10_200']

1.5020515709327509

**ID_20000_200_700**
0.0/25.0 points (graded)  
Following the PA5 definition, what value does ID_20000_200_700 have?  
**Answer:1.2150732884765727**   
**+/- 15% range:1.0328122952050869, 1.3973342817480585**


In [101]:
res['ID_20000_200_700']

1.237115505288337

**ID_20000_700_2000**
0.0/25.0 points (graded)  
Following the PA5 definition, what value does ID_20000_200_700 have?  
**Answer:1.1196299391601852**   
**+/- 15% range:0.9516854482861574, 1.287574430034213**


In [102]:
res['ID_20000_700_2000']

1.0997170517303632